In [30]:
from bs4 import BeautifulSoup
import requests
from langdetect import detect
import pandas as pd
from datetime import date

In [31]:
def get_soup(url):
    response =  requests.get(url)
    response.content
    soup = BeautifulSoup(response.content)
    return soup

In [32]:
def get_titles(soup):
    titles = soup.find_all("h2",attrs={"class":"card__job-title"})
    title_list=[title.text.strip() for title in titles]
    return title_list


In [33]:
def get_location(soup):
    locations= soup.find_all("div",attrs={"class":"card__job-location"})

    loc_list=[loc.text.strip() for loc in locations]
    return loc_list

In [34]:
def get_text_lan_page(soup):
    snippet= soup.find_all("p",attrs={"class":"card__job-snippet"})

    desc_list=[desc.text.strip() for desc in snippet]
    lan_list=[detect(text) for text in desc_list]
    page_list=[page for desc in snippet]
    return desc_list, lan_list,page_list


In [35]:
def get_company(soup):
    companies= soup.find_all("div",attrs={"class":"card__job-empname-label"})

    comp_list=[comp.text.strip() for comp in companies]
    return comp_list

In [36]:
## GET THE LINKS
def get_link(soup):
    div=soup.find_all("div",attrs={"class":"link-job-wrap"})
    links= ["https://nl.talent.com"+link.get("data-link") for link in div]
    return links

In [37]:
## CHECK INITIAL PAGE NUMBER
page=1
url= f"https://nl.talent.com/jobs?k=%26%2334%3BJunior+data%26%2334%3B&l=amsterdam&date=7d&radius=15&p={page}&context=serp_pagination"

soup=get_soup(url)
page_num=soup.find_all("span",attrs={"class":"page-number"})

page_list=[page.text.strip() for page in page_num]
page_list

['1', '2']

In [38]:
## CHECK TOTAL NUMBER OF PAGE RESULTS

if len(page_list)>1:
    page=2
    while len(page_list)>=page:
    
        url= f"https://nl.talent.com/jobs?k=%26%2334%3BJunior+data%26%2334%3B&l=amsterdam&date=7d&radius=15&p={page}&context=serp_pagination"
        response =  requests.get(url)
        response.content
        soup = BeautifulSoup(response.content)
        page_num=soup.find_all("span",attrs={"class":"page-number"})

        page_list=[page.text.strip() for page in page_num]
        #print(page_list)
        if len(page_list)>page:
            page=page+1
            #print('here')
        else:
            
            break
    print(page_list)
    print(len(page_list))
else:
    pass

['2', '3', '4', '5', '6']
5


In [39]:
title_list=[]
loc_list=[]
desc_list=[]
lan_list=[]
page_num_list=[]
links_list=[]
comp_list=[]
for page in page_list:
    url= f"https://nl.talent.com/jobs?k=%26%2334%3BJunior+data%26%2334%3B&l=amsterdam&date=7d&radius=15&p={page}&context=serp_pagination"
    soup=get_soup(url)
    
    #titles=get_titles(soup)
    title_list=title_list + get_titles(soup)
    #print(title_list)
    loc_list=loc_list+ get_location(soup)
    ads , lan, pag = get_text_lan_page(soup)
    desc_list=desc_list+ ads
    lan_list = lan_list + lan
    page_num_list=page_num_list+pag
    links_list=links_list+get_link(soup)
    comp_list=comp_list+get_company(soup)



In [40]:
print(len(title_list))
print(len(loc_list))
print(len(desc_list))
print(len(lan_list))
print(len(page_num_list))
print(len(links_list))

129
129
129
129
129
129


In [42]:
## CREATE DATAFRAME

df_jobs=pd.DataFrame(list(zip(title_list,comp_list,loc_list,desc_list,lan_list,links_list)), columns =['job_title', 'company_name','company_location','job_desc','lan','link'])

df_jobs

,job_title,company_name,company_location,job_desc,lan,link
0,Junior Account Manager Breda,Manpower,"Amsterdam, Noord-Holland",vandaag nog! Manpower is voor een opdrachtgeve...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
1,Junior Accountmanager Binnendienst,Actief Werkt!,"Amsterdam, Noord-Holland",Wil jij werken als accountmanager binnendienst...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
2,Parttime junior marketeer,Unique,"Amsterdam, Noord-Holland",Over de functieBen jij helemaal thuis in de di...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
3,Junior manager ARTIS-plein (Ref. 2523),Vermaat Groep B.V.,"Amsterdam, Noord-Holland",Taken en verantwoordelijkhedenHey jonge ondern...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
4,JUNIOR ENGINEER,Stedin,"Amsterdam, Noord-Holland",We stappen over op elektrische auto's. Wekken ...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
...,...,...,...,...,...,...
124,Junior Sous ChefGeplaatst op: 02-01-2023,Ron Gastrobar Amsterdam,"Amsterdam, Noord-Holland",Wat voor jou!Bij Ron Gastrobar in Amsterdam zi...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
125,Junior Sous ChefGeplaatst op: 29-11-2022,Ron Gastrobar Oriental Downtown,"Amsterdam, Noord-Holland",Wat voor jou!Bij Ron Gastrobar in Amsterdam zi...,nl,https://nl.talent.com/redirect?l=amsterdam&dat...
126,Data Protection Officer,Exact,"Amsterdam, Noord-Holland",Senior Professional Job Delft Corporate Functi...,en,https://nl.talent.com/redirect?l=amsterdam&dat...
127,BI (business intelligence) / data specialist,ARQ Nationaal Psychotrauma Centrum,"Diemen, Noord-Holland",Wat ga je doen?Als BI (business intelligence)....,nl,https://nl.talent.com/redirect?l=amsterdam&dat...


In [43]:
# EXPORT DATAFRAME TO EXCEL
date=date.today()
filename=f"Excel_files/talent_juniordata_{date}.xlsx"
df_jobs.to_excel(filename,index=False)

In [ ]:
#filtered_df=df_jobs[(~df_jobs["Title"].str.contains("senior|Senior"))& (df_jobs["Lan"]=="en")]

In [ ]:
# filename_filter=f"Excel_files/FILTERED_talent_juniordata_{date}.xlsx"
# filtered_df.to_excel(filename_filter,index=False)
